In [1]:

#Importing libraries
import requests
import os
import json
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup  
import time
import numpy as np
import requests
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import traceback
from webdriver_manager.chrome import ChromeDriverManager  


In [3]:

options = webdriver.ChromeOptions()
driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
driver.set_page_load_timeout(20)



Trying to download new driver from http://chromedriver.storage.googleapis.com/83.0.4103.39/chromedriver_win32.zip
Unpack archive C:\Users\venub\.wdm\drivers\chromedriver\83.0.4103.39\win32\chromedriver.zip


C:\Users\venub\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  


In [ ]:
Username: plaksiy2
Password: ru9hphkh


In [4]:
links=['https://www-sciencedirect-com.dcu.idm.oclc.org/journal/journal-of-corporate-finance/issues?page=2']


In [5]:

def get_url_and_wait_for_page_load(_driver, url):
    _driver.get(url)
    WebDriverWait(driver, 30).until(
        EC.visibility_of_all_elements_located((By.XPATH, "//div[@class = 'usabilla_live_button_container']")))


In [7]:
volume_links, issue_parse_failed = [], []  # you can reprocess failed list if you wish
for link in links:
    try:
        print ("Looking for volume links @ {}".format(link))
        volume_links += [link.get_attribute("href")
                         for link in driver.find_elements_by_xpath("//a[contains(@class, 'js-issue-item-link')]")]
        print ("Total volume links: {}".format(len(volume_links)))
    except:
        print ("Failed to get volume links @ {}".format(link))
        issue_parse_failed.append(link)

Looking for volume links @ https://www-sciencedirect-com.dcu.idm.oclc.org/journal/journal-of-corporate-finance/issues?page=2
Total volume links: 4


In [8]:
article_links, volume_parse_failed = [], []  # you can reprocess failed list if you wish
for volume_link in volume_links:
    try:
        print ("Looking for article links @ {}".format(volume_link))
        get_url_and_wait_for_page_load(driver, volume_link)
        article_links += [link.get_attribute("href")
                          for link in driver.find_elements_by_xpath("//a[contains(@class, 'article-content-title')]")]
        print ("Total article links: {}".format(len(article_links)))
    except:
        
        print ("Failed to get article links @ {}".format(volume_link))
        volume_parse_failed.append(volume_link)

Looking for article links @ https://www-sciencedirect-com.dcu.idm.oclc.org/journal/journal-of-corporate-finance/vol/64/suppl/C
Total article links: 28
Looking for article links @ https://www-sciencedirect-com.dcu.idm.oclc.org/journal/journal-of-corporate-finance/vol/62/suppl/C
Total article links: 80
Looking for article links @ https://www-sciencedirect-com.dcu.idm.oclc.org/journal/journal-of-corporate-finance/vol/61/suppl/C
Total article links: 93
Looking for article links @ https://www-sciencedirect-com.dcu.idm.oclc.org/journal/journal-of-corporate-finance/vol/60/suppl/C
Total article links: 134


In [9]:
options = webdriver.ChromeOptions() 



In [10]:
links_new=article_links[63:]

In [73]:
driver.execut_script("window.scrollTo(0,1000)")

AttributeError: 'WebDriver' object has no attribute 'execut_script'

In [11]:
options.add_argument("start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

In [12]:
driver = webdriver.Chrome(options=options, executable_path=r"C:/selenium/chromedriver.exe")


In [44]:
driver.switch_to_window(driver.window_handles[0])

C:\Users\venub\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.window instead
  """Entry point for launching an IPython kernel.


In [13]:
corporate_finance_list=[]

In [15]:
driver.execute_script("window.scrollTo(0,700)")

In [14]:
for link in article_links:
    driver.get(link)
    window0  = driver.current_window_handle
    sa={}
    sleep(7)
    try:
        sa['title']=driver.find_element_by_xpath('//*[@id="screen-reader-main-title"]/span').text
        sa["link"]=driver.find_element_by_xpath('//*[@id="doi-link"]/a[1]').text
        sa["publishment_details"]=driver.find_element_by_xpath('//*[@id="publication"]/div[2]/div').text
        sa["abstract"]=driver.find_element_by_xpath('/html/body/div[4]/div/div/section/div/div/div/div[2]/article/div[5]/div').text
        sa["highlights"]=driver.find_element_by_xpath('//*[@id="ab0010"]').text

        
        sa["citations"]=driver.find_element_by_xpath('//*[@id="citing-articles-header"]').text
    except:
        print("elements missing")
    driver.execute_script("window.scrollTo(0,400);")
    try:
        sa["figures"]=driver.find_element_by_xpath('//*[@id="toc-figures"]/h2').text
        sa["tables"]=driver.find_element_by_xpath('//*[@id="toc-tables"]/h2').text
    except:
        print("elements missing")
    try:
        sa["references"]=driver.find_element_by_xpath('/html/body/div/div/div/section/div/div/div/div[2]/article/section[3]').text
    except:
        print("references not found")
    driver.execute_script("window.scrollTo(400,700)")
    try:
        sa["body"]= WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="body"]'))).text
    except:
        print("body not found")
    try:
        sa["k/c"]=driver.find_element_by_xpath('/html/body/div[4]/div/div/section/div/div/div/div[2]/article/div[6]/div[1]').text
        sa["c/k"]=driver.find_element_by_xpath('/html/body/div[4]/div/div/section/div/div/div/div[2]/article/div[6]/div[2]').text
    except:
        print("classifications missing")
        
    driver.execute_script("window.scrollTo(700,0);")
    try:
        driver.find_element_by_xpath('//*[@id="author-group"]/a[1]/span').click()
        sa["author1"]=driver.find_element_by_xpath('//*[@id="workspace-author"]/div[1]').text
        sleep(2)
        iframe = driver.find_element_by_xpath('//iframe[contains(@title, "Author Profile")]')
        driver.switch_to.frame(iframe)
        sa["author1-h-index"]=driver.find_element_by_xpath('//*[@id="profile-preview-page"]/div/div/div/div[1]/div[3]/ul/li[1]/data').text
        sa["author1-citations"]=driver.find_element_by_xpath('//*[@id="profile-preview-page"]/div/div/div/div[1]/div[3]/ul/li[2]/data').text
        sa["author1-university"]=driver.find_element_by_xpath('//*[@id="profile-preview-page"]/div/div/div/div[1]/div[2]/span').text
        driver.switch_to_window(window0)
    except:
        print("no author1")
    try:
        driver.find_element_by_xpath('//*[@id="author-group"]/a[2]/span').click()
        sa["author2"]=driver.find_element_by_xpath('//*[@id="workspace-author"]/div[1]').text
        sleep(2)
        iframe = driver.find_element_by_xpath('//iframe[contains(@title, "Author Profile")]')
        driver.switch_to.frame(iframe)
        sa["author2-h-index"]=driver.find_element_by_xpath('//*[@id="profile-preview-page"]/div/div/div/div[1]/div[3]/ul/li[1]/data').text
        sa["author2-citations"]=driver.find_element_by_xpath('//*[@id="profile-preview-page"]/div/div/div/div[1]/div[3]/ul/li[2]/data').text
        sa["author2-university"]=driver.find_element_by_xpath('//*[@id="profile-preview-page"]/div/div/div/div[1]/div[2]/span').text
        driver.switch_to_window(window0)
    except:
        print("no author2")
    try:
        driver.find_element_by_xpath('//*[@id="author-group"]/a[3]/span').click()
        sa["author3"]=driver.find_element_by_xpath('//*[@id="workspace-author"]/div[1]').text
        sleep(2)
        iframe = driver.find_element_by_xpath('//iframe[contains(@title, "Author Profile")]')
        driver.switch_to.frame(iframe)
        sa["author3-h-index"]=driver.find_element_by_xpath('//*[@id="profile-preview-page"]/div/div/div/div[1]/div[3]/ul/li[1]/data').text
        sa["author3-citations"]=driver.find_element_by_xpath('//*[@id="profile-preview-page"]/div/div/div/div[1]/div[3]/ul/li[2]/data').text
        sa["author3-university"]=driver.find_element_by_xpath('//*[@id="profile-preview-page"]/div/div/div/div[1]/div[2]/span').text
        driver.switch_to_window(window0)
    except:
        print("no author3")
    try:
        driver.find_element_by_xpath('//*[@id="author-group"]/a[4]/span').click()
        sa["author4"]=driver.find_element_by_xpath('//*[@id="workspace-author"]/div[1]').text
        sleep(2)
        iframe = driver.find_element_by_xpath('//iframe[contains(@title, "Author Profile")]')
        driver.switch_to.frame(iframe)
        sa["author4-h-index"]=driver.find_element_by_xpath('//*[@id="profile-preview-page"]/div/div/div/div[1]/div[3]/ul/li[1]/data').text
        sa["author4-citations"]=driver.find_element_by_xpath('//*[@id="profile-preview-page"]/div/div/div/div[1]/div[3]/ul/li[2]/data').text
        sa["author4-university"]=driver.find_element_by_xpath('//*[@id="profile-preview-page"]/div/div/div/div[1]/div[2]/span').text
        driver.switch_to_window(window0)
    except:
        print("no author4")
    try:
        driver.find_element_by_xpath('//*[@id="author-group"]/a[5]/span').click()
        sa["author5"]=driver.find_element_by_xpath('//*[@id="workspace-author"]/div[1]').text
        sleep(2)
        iframe = driver.find_element_by_xpath('//iframe[contains(@title, "Author Profile")]')
        driver.switch_to.frame(iframe)
        sa["author5-h-index"]=driver.find_element_by_xpath('//*[@id="profile-preview-page"]/div/div/div/div[1]/div[3]/ul/li[1]/data').text
        sa["author5-citations"]=driver.find_element_by_xpath('//*[@id="profile-preview-page"]/div/div/div/div[1]/div[3]/ul/li[2]/data').text
        sa["author5-university"]=driver.find_element_by_xpath('//*[@id="profile-preview-page"]/div/div/div/div[1]/div[2]/span').text
        driver.switch_to_window(window0)
    except:
        print("no author5")
    corporate_finance_list.append(sa)
    


C:\Users\venub\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: DeprecationWarning: use driver.switch_to.window instead
C:\Users\venub\Anaconda3\lib\site-packages\ipykernel_launcher.py:60: DeprecationWarning: use driver.switch_to.window instead
C:\Users\venub\Anaconda3\lib\site-packages\ipykernel_launcher.py:72: DeprecationWarning: use driver.switch_to.window instead


no author4
no author5
no author2
no author3
no author4
no author5


KeyboardInterrupt: 

In [15]:
corporate_finance_list

[{'title': 'Co-opted directors, covenant intensity, and covenant violations',
  'link': 'https://doi.org/10.1016/j.jcorpfin.2020.101628',
  'publishment_details': 'Volume 64, October 2020, 101628',
  'abstract': 'Highlights\n•\nCreditors (banks) view co-opted directors as weaker monitors.\n•\nBorrowers with more co-opted boards face more loan covenant restrictions.\n•\nBorrowers with more co-opted boards are more likely to violate loan covenants.\n•\nNon-co-opted independent directors appear to be the most effective monitors.',
  'highlights': 'Highlights\n•\nCreditors (banks) view co-opted directors as weaker monitors.\n•\nBorrowers with more co-opted boards face more loan covenant restrictions.\n•\nBorrowers with more co-opted boards are more likely to violate loan covenants.\n•\nNon-co-opted independent directors appear to be the most effective monitors.',
  'citations': 'Citing articles (0)',
  'figures': 'Figures (1)',
  'tables': 'Tables (12)',
  'references': 'References\nAdams 

In [16]:
corporate_finance=pd.DataFrame(corporate_finance_list)

In [17]:
corporate_finance['tables']=corporate_finance['tables'].str.extract('(\d+)')
corporate_finance['figures']=corporate_finance['figures'].str.extract('(\d+)')
a=corporate_finance['publishment_details'].str.split(',',expand=True)
corporate_finance['volume']=a[0]
corporate_finance['published time']=a[1]
corporate_finance['article number']=a[2]
intro=corporate_finance['body'].str.split('\n2. ',expand=True)
corporate_finance["introduction"]=intro[0]
corporate_finance["Citations"]=corporate_finance['citations'].str.extract('(\d+)')

In [18]:
corporate_finance.columns

Index(['title', 'link', 'publishment_details', 'abstract', 'highlights',
       'citations', 'figures', 'tables', 'references', 'body', 'k/c', 'c/k',
       'author1', 'author1-h-index', 'author1-citations', 'author1-university',
       'author2', 'author2-h-index', 'author2-citations', 'author2-university',
       'author3', 'author3-h-index', 'author3-citations', 'author3-university',
       'volume', 'published time', 'article number', 'introduction',
       'Citations'],
      dtype='object')

In [108]:
body=corporate_finance['body']

In [19]:
corporate_finance

,title,link,publishment_details,abstract,highlights,citations,figures,tables,references,body,...,author2-university,author3,author3-h-index,author3-citations,author3-university,volume,published time,article number,introduction,Citations
0,"Co-opted directors, covenant intensity, and co...",https://doi.org/10.1016/j.jcorpfin.2020.101628,"Volume 64, October 2020, 101628",Highlights\n•\nCreditors (banks) view co-opted...,Highlights\n•\nCreditors (banks) view co-opted...,Citing articles (0),1,12,"References\nAdams and Ferreira, 2007\nR.B. Ada...",1. Introduction\nCo-opted directors are those ...,...,Monash Business School,Tram Vu,3,15,Monash Business School,Volume 64,October 2020,101628,1. Introduction\nCo-opted directors are those ...,0
1,Do stress tests affect bank liquidity creation?,https://doi.org/10.1016/j.jcorpfin.2020.101622,"Volume 64, October 2020, 101622",Highlights\n•\nWe examine the impact of Federa...,Highlights\n•\nWe examine the impact of Federa...,Citing articles (0),3,19,"References\nAcharya and Naqvi, 2012\nV. Achary...",1. Introduction\nBank capital acts as a buffer...,...,NaN,NaN,NaN,NaN,NaN,Volume 64,October 2020,101622,1. Introduction\nBank capital acts as a buffer...,0


In [109]:
body

0                                                    NaN
1      “Obviously, if shareholders (or board of direc...
2      1. Introduction\nFamily business owners who wi...
3      1. Introduction\nIn this paper, I study the ca...
4      “Large, creditworthy buyers force longer payme...
                             ...                        
450    1. Introduction\nIn his influential work, Mert...
451    1. Introduction\nAcharya and Pedersen (2005), ...
452    1. Introduction\nFollowing the sub-prime crisi...
453    1. Introduction\nAcquisitions are among the mo...
454    1. Introduction\nTwo recent trends in corporat...
Name: body, Length: 455, dtype: object

In [81]:
C1=corporate_finance["body"].str.split('Conclusion\n',expand=True)

In [82]:
C1 = C1.replace(np.nan, '', regex=True)

In [83]:
C1

,0,1
0,,
1,1. Introduction\nThis study investigates how c...,
2,1. Introduction\nThe literature on agency prob...,In this paper we examine how equity-based comp...
3,1. Introduction\nDespite the fact that the gen...,"In this paper, we conducted a comprehensive an..."
4,1. Introduction\nMany publicly traded corporat...,Our study of the value and likelihood of acqui...
...,...,...
380,1. Introduction\nThe relation between sharehol...,"To date, there has been little modeling of the..."
381,1. Introduction\nPrices in well-functioning ma...,
382,1. Introduction\nSubsidiaries of multinational...,We analyze an optimal tax management strategy ...
383,,


In [84]:
C2=corporate_finance["body"].str.split('Conclusions\n',expand=True)

In [85]:
C2 = C2.replace(np.nan, '', regex=True)

In [86]:
C2

0                                                       
1      This paper investigates sunset provisions of e...
2                                                       
3                                                       
4                                                       
                             ...                        
380                                                     
381    Interpreting the Stiglitz and Weiss (1981)mode...
382                                                     
383                                                     
384                                                     
Name: 1, Length: 385, dtype: object

In [87]:
corporate_finance["Conclusion"]=C1[1].str.cat(C2[1],sep="")

In [88]:
corporate_finance['Conclusion']

0                                                       
1      This paper investigates sunset provisions of e...
2      In this paper we examine how equity-based comp...
3      In this paper, we conducted a comprehensive an...
4      Our study of the value and likelihood of acqui...
                             ...                        
380    To date, there has been little modeling of the...
381    Interpreting the Stiglitz and Weiss (1981)mode...
382    We analyze an optimal tax management strategy ...
383                                                     
384                                                     
Name: Conclusion, Length: 385, dtype: object

In [89]:
corporate_finance

,title,link,publishment_details,abstract,figures,tables,references,body,k/c,c/k,...,author4-university,author5,author5-h-index,author5-citations,author5-university,volume,published time,article number,introduction,Conclusion
0,Editorial Board,https://doi.org/10.1016/S0929-1199(08)00103-X,"Volume 14, Issue 5, December 2008, Page IFC",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Volume 14,Issue 5,December 2008,NaN,
1,You can't take it with you: Sunset provisions ...,https://doi.org/10.1016/j.jcorpfin.2008.08.006,"Volume 14, Issue 5, December 2008, Pages 499-511",Abstract\nCompany stock option plans have dive...,3,7,"References\nAmmann and Seiz, 2004\nAmmann M., ...",1. Introduction\nThis study investigates how c...,JEL classification\nG32J33,Keywords\nExecutive stock optionsVestingForfei...,...,NaN,NaN,NaN,NaN,NaN,Volume 14,Issue 5,December 2008,1. Introduction\nThis study investigates how c...,This paper investigates sunset provisions of e...
2,Board compensation practices and agency costs ...,https://doi.org/10.1016/j.jcorpfin.2008.09.004,"Volume 14, Issue 5, December 2008, Pages 512-531",Abstract\nExtant theory and empirical evidence...,NaN,NaN,"References\nAltman, 1968\nAltman E.\nFinancial...",1. Introduction\nThe literature on agency prob...,JEL classification\nG34G32J33,Keywords\nDirector incentivesCost of debtCorpo...,...,NaN,NaN,NaN,NaN,NaN,Volume 14,Issue 5,December 2008,1. Introduction\nThe literature on agency prob...,In this paper we examine how equity-based comp...
3,The determinants of board composition in a tra...,https://doi.org/10.1016/j.jcorpfin.2008.09.005,"Volume 14, Issue 5, December 2008, Pages 532-549",NaN,2,9,"References\nAbe, 2003\nAbe N.\nKeieisha turnov...",1. Introduction\nDespite the fact that the gen...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Volume 14,Issue 5,December 2008,1. Introduction\nDespite the fact that the gen...,"In this paper, we conducted a comprehensive an..."
4,Protective governance choices and the value of...,https://doi.org/10.1016/j.jcorpfin.2008.09.011,"Volume 14, Issue 5, December 2008, Pages 550-566",NaN,NaN,NaN,"References\nAdams and Ferreira, 2007\nAdams R....",1. Introduction\nMany publicly traded corporat...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Volume 14,Issue 5,December 2008,1. Introduction\nMany publicly traded corporat...,Our study of the value and likelihood of acqui...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,Board effectiveness and board dissent: A model...,https://doi.org/10.1016/S0929-1199(97)00009-6,"Volume 4, Issue 1, March 1998, Pages 53-70","Abstract\nTo date, there has been little model...",NaN,NaN,"References\nBerle and Means, 1932\nBerle, A.A....",1. Introduction\nThe relation between sharehol...,Keywords\nBoards of directorsManagerial discip...,JEL classification\nG32,...,NaN,NaN,NaN,NaN,NaN,Volume 4,Issue 1,March 1998,1. Introduction\nThe relation between sharehol...,"To date, there has been little modeling of the..."
381,An options-based model of equilibrium credit r...,https://doi.org/10.1016/S0929-1199(97)00010-2,"Volume 4, Issue 1, March 1998, Pages 71-85",Abstract\nThis paper applies options theory to...,5,1,"References\nBerger and Udell, 1992\nA.N Berger...",1. Introduction\nPrices in well-functioning ma...,Keywords\nEquilibrium credit rationingOption p...,JEL classification\nD82G13G30,...,NaN,NaN,NaN,NaN,NaN,Volume 4,Issue 1,March 1998,1. Introduction\nPrices in well-functioning ma...,Interpreting the Stiglitz and Weiss (1981)mode...
382,Internal financing of multinational subsidiari...,https://doi.org/10.1016/S0929-1199(97)00011-4,"Volume 4, Issue 1, March 1998, Pages 87-106",Abstract\nMultinational subsidiaries are gener...,1,NaN,"References\nChowdhry and Nanda, 1994\nB. Chowd...",1. Introduction\nSubsidiaries of multinational...,Keywords\nMultinational capital structureInter...,JEL classification\nG3G32F3F30,...,NaN,NaN,NaN,NaN,NaN,Volume 4,Issue 1,March 1998,1. Introduction\nSubsidiaries of multination

In [90]:
corporate_finance.to_csv("Corporatefinance14.csv")

In [78]:
511+115


626

In [10]:
ab=pd.read_csv("jocf1.csv")

In [12]:
ab

,Unnamed: 0,title,link,publishment_details,abstract,highlights,k/c,c/k,body,author1,...,author4-citations,author4-university,author5,author5-h-index,author5-citations,author5-university,volume,published time,article number,introduction
0,0,"Hard markets, hard times: On the inefficiency ...",https://doi.org/10.1016/j.jcorpfin.2019.101553,"Volume 62, June 2020, 101553",Abstract\nThis study explores the existence of...,Highlights\n•\nWe find that the sponsor's expe...,JEL classification\nG12G14G22G32,Keywords\nCAT bondHard marketMarket inefficien...,1. Introduction\nRisk management is of central...,Tobias Götze,...,NaN,NaN,NaN,NaN,NaN,NaN,Volume 62,June 2020,101553,1. Introduction\nRisk management is of central...
1,1,How legal and institutional environments shape...,https://doi.org/10.1016/j.jcorpfin.2019.101555,"Volume 62, June 2020, 101555",Abstract\nI investigate how legal and institut...,Highlights\n•\nInitial legal and institutional...,Keywords\nLegal systemsInstitutional environme...,JEL classification\nG21G24G32G34,1. Introduction\nAccording to the “law & finan...,Christophe J. Godlewski,...,NaN,NaN,NaN,NaN,NaN,NaN,Volume 62,June 2020,101555,1. Introduction\nAccording to the “law & finan...
2,2,Repurchases after being well known as good news,https://doi.org/10.1016/j.jcorpfin.2019.101552,"Volume 62, June 2020, 101552","Abstract\nUsing recent U.S. data, we find that...",Highlights\n•\nLong-term performance of buybac...,JEL classification\nG30G32G35,Keywords\nShare repurchasesPayout policies,Loading...,Inmoo Lee,...,NaN,NaN,NaN,NaN,NaN,NaN,Volume 62,June 2020,101552,Loading...
3,3,Dual agency problems in family firms: Evidence...,https://doi.org/10.1016/j.jcorpfin.2019.101556,"Volume 62, June 2020, 101556",Abstract\nWe use director elections to analyze...,Highlights\n•\nFounding family ownership resul...,JEL classification\nG30G34,Keywords\nDirector electionsFamily firmsGovern...,Investors holding more than a quarter of the s...,Rasha Ashraf,...,NaN,NaN,NaN,NaN,NaN,NaN,Volume 62,June 2020,101556,Investors holding more than a quarter of the s...
4,4,Do political connection disruptions increase l...,https://doi.org/10.1016/j.jcorpfin.2019.101554,"Volume 62, June 2020, 101554",Abstract\nThis paper investigates whether the ...,Highlights\n•\nUsing an exogeneous shock in Ch...,JEL classifications\nG38J21J30P26,Keywords\nPolitical connection disruptionsLabo...,1. Introduction\nThe impact of political conne...,Chunyan Wei,...,NaN,NaN,NaN,NaN,NaN,NaN,Volume 62,June 2020,101554,1. Introduction\nThe impact of political conne...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,109,Debt covenants and corporate acquisitions,https://doi.org/10.1016/j.jcorpfin.2018.10.008,"Volume 53, December 2018, Pages 174-201",Abstract\nWe investigate the impact of debt co...,Highlights\n•\nAcquirers with covenants pay lo...,JEL classification\nG34G32,Keywords\nAcquisitionsDebt covenantsControl ri...,1. Introduction\nCovenants have typically been...,Mai M. Daher,...,NaN,NaN,NaN,NaN,NaN,NaN,Volume 53,December 2018,Pages 174-201,1. Introduction\nCovenants have typically been...
110,110,Sovereign wealth funds and equity pricing: Evi...,https://doi.org/10.1016/j.jcorpfin.2018.10.007,"Volume 53, December 2018, Pages 202-224",Abstract\nWe investigate the impact of soverei...,Highlights\n•\nWe examine the impact of sovere...,NaN,NaN,NaN,Sabri Boubaker,...,11,Université de Sousse,NaN,NaN,NaN,NaN,Volume 53,December 2018,Pages 202-224,NaN
111,111,Shareholder approval thresholds in acquisition...,https://doi.org/10.1016/j.jcorpfin.2018.10.005,"Volume 53, December 2018, Pages 225-245",Abstract\nWe exploit a 2013 Delaware law that ...,Highlights\n•\nA Delaware law in 2013 reduced ...,JEL classification\nG32G34G38,Keywords\nShareholder votingTender offerMerger...,1. Introduction\nConsiderable debate exists on...,Audra,...,NaN,NaN,NaN,NaN,NaN,NaN,Volume 53,December 2018,Pages 225-245,1. Introduction\nConsiderable debate exis

In [3]:
body=ab['body'].head(8)

In [5]:
body

0    1. Introduction\nCorporate board structure and...
1    1. Introduction\nIt is the consensus in the li...
2    1. Introduction\nCorporate managers are assume...
3    1. Introduction\nIt is widely recognized that ...
4    1. Introduction\nInternational equity flows ha...
5                                                  NaN
6                                                  NaN
7    1. Introduction\nIn the U.S., operating leases...
Name: body, dtype: object

In [9]:
c=body.str.split('Conclusion\n', expand=True)

In [10]:
c

,0,1
0,1. Introduction\nCorporate board structure and...,"In this paper, we use a panel data set coverin..."
1,1. Introduction\nIt is the consensus in the li...,The M&A literature document that a non-negligi...
2,1. Introduction\nCorporate managers are assume...,None
3,1. Introduction\nIt is widely recognized that ...,A large literature on industrial diversificati...
4,1. Introduction\nInternational equity flows ha...,We offer a political economy explanation for t...
5,NaN,NaN
6,NaN,NaN
7,"1. Introduction\nIn the U.S., operating leases...",None


In [11]:
d=body.str.split('Conclusions\n', expand=True)

In [12]:
d

,0,1
0,1. Introduction\nCorporate board structure and...,None
1,1. Introduction\nIt is the consensus in the li...,None
2,1. Introduction\nCorporate managers are assume...,We address a manager–owner agency aspect that ...
3,1. Introduction\nIt is widely recognized that ...,None
4,1. Introduction\nInternational equity flows ha...,None
5,NaN,NaN
6,NaN,NaN
7,"1. Introduction\nIn the U.S., operating leases...",U.S. firms lease a substantial amount of asset...


In [ ]:
df['Name'] = df['First'].str.cat(df['Last'], sep =" ") 

In [15]:
C1=ab["body"].str.split('Conclusion\n',expand=True)

In [ ]:
C1 = C1.replace(np.nan, '', regex=True)

In [23]:
C1

,0,1
0,1. Introduction\nCorporate board structure and...,"In this paper, we use a panel data set coverin..."
1,1. Introduction\nIt is the consensus in the li...,The M&A literature document that a non-negligi...
2,1. Introduction\nCorporate managers are assume...,
3,1. Introduction\nIt is widely recognized that ...,A large literature on industrial diversificati...
4,1. Introduction\nInternational equity flows ha...,We offer a political economy explanation for t...
...,...,...
753,,
754,,
755,,
756,,


In [24]:
C2=ab["body"].str.split('Conclusions\n',expand=True)

In [25]:
C2 = C2.replace(np.nan, '', regex=True)

In [26]:
C2

,0,1
0,1. Introduction\nCorporate board structure and...,
1,1. Introduction\nIt is the consensus in the li...,
2,1. Introduction\nCorporate managers are assume...,We address a manager–owner agency aspect that ...
3,1. Introduction\nIt is widely recognized that ...,
4,1. Introduction\nInternational equity flows ha...,
...,...,...
753,,
754,,
755,,
756,,


In [27]:
ab["Conclusion"]=C1[1].str.cat(C2[1],sep="")

In [28]:
ab["Conclusion"]

0      In this paper, we use a panel data set coverin...
1      The M&A literature document that a non-negligi...
2      We address a manager–owner agency aspect that ...
3      A large literature on industrial diversificati...
4      We offer a political economy explanation for t...
                             ...                        
753                                                     
754                                                     
755                                                     
756                                                     
757                                                     
Name: Conclusion, Length: 758, dtype: object

In [ ]:
//*[@id="bi0005"]
//*[@id="bi0005"]
//*[@id="aep-bibliography-id32"]
//*[@id="bi0005"]

In [ ]:
/html/body/div[4]/div/div/section/div/div/div/div[2]/article/section[3]
/html/body/div[4]/div/div/section/div/div/div/div[2]/article/section[3]
/html/body/div[3]/div/div/section/div/div/div/div[2]/article/section[3]
/html/body/div[4]/div/div/section/div/div/div/div[2]/article/section[3]